In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
import boto3
import logging

In [ ]:
# Configuração do Logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
# Função para gerar dados fictícios
def gerar_dados_vendas(num_linhas):
    produtos = ["Laptop", "Smartphone", "Tablet", "Monitor", "Keyboard", "Mouse", "Headphones", "Printer"]
    data_inicial = datetime(2024, 11, 1)
    
    dados = []
    logger.info("Iniciando a geração de dados fictícios.")
    for i in range(1, num_linhas + 1):
        sale_id = i
        product = random.choice(produtos)
        amount = round(random.uniform(50.0, 2000.0), 2)  # Valor entre 50 e 2000
        sale_date = (data_inicial + timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d")
        dados.append({"sale_id": sale_id, "product": product, "amount": amount, "sale_date": sale_date})
        
        if i % (num_linhas // 10) == 0:  # Log a cada 10% dos dados gerados
            logger.info(f"{i} de {num_linhas} linhas geradas.")
    
    logger.info("Geração de dados concluída.")
    return pd.DataFrame(dados)

In [ ]:
# Configurações do MinIO
# TODO: Informar endpoint do MinIO, a chave de acesso e a chave secreta.
minio_config = {
    "endpoint_url": "http://vmi2122960.contaboserver.net:9000",  # Atualize conforme necessário
    "aws_access_key_id": "admin",
    "aws_secret_access_key": "password",
}

In [ ]:
# TODO: Informar o nome do bucket onde está os arquivos. Exemplo: 'prd-data-raw'
bucket_name = "prd-data-raw"
csv_file_name = "vendas.csv"

In [ ]:
try:
    # Gerar DataFrame
    num_linhas = 1000000
    logger.info(f"Gerando DataFrame com {num_linhas} linhas.")
    df = gerar_dados_vendas(num_linhas)
    
    # Salvar DataFrame como CSV em memória
    logger.info("Convertendo DataFrame para CSV em memória.")
    csv_buffer = df.to_csv(index=False)
    
    # Criar cliente boto3 para o MinIO
    logger.info("Conectando ao MinIO com as configurações fornecidas.")
    s3_client = boto3.client("s3", **minio_config)
    
    # Fazer upload do CSV para o MinIO
    logger.info(f"Fazendo upload do arquivo CSV para o bucket '{bucket_name}' no MinIO.")
    s3_client.put_object(
        Bucket=bucket_name,
        Key=f"vendas/{csv_file_name}",
        Body=csv_buffer,
        ContentType="text/csv"
    )
    logger.info(f"Upload concluído: {csv_file_name} no bucket '{bucket_name}'.")
except Exception as e:
    logger.error(f"Erro durante o processo: {e}")